# Study C
This notebook shows the full workflow for building models, simulating growth and obtaining SCFA predictions from data collected by the _ex vivo_ study conducted by the Gurry Lab in 2021 (Study C)

In [ ]:
import pandas as pd
import micom
import micom.measures
from plotnine import *
import scipy

%matplotlib inline

## SCFA Flux
Here we will gather the net SCFA production across the culturing period. The "Measured" columns contain the measured flux over the course of the experiment (mM/h, [T1]-[T0]/h), while the "Baseline" columns contain the T0 concentrations (mM)

In [ ]:
scfa = pd.read_csv('../data/raw_data/studyC_original_scfas.csv',index_col = 0)

scfa['donor'] = scfa.index.str.split('_').str[0]
scfa['treatment'] = scfa.index.str.split('_').str[1]

scfa = scfa.groupby(['donor','treatment']).mean().reset_index()

scfa = scfa[scfa['treatment']!='inulin+pectin']

scfa['sample_id'] = scfa['donor']+'_'+scfa['treatment']
scfa.set_index('sample_id', inplace = True)
scfa

## Taxonomy Table
Next, we will read in abundance data for all samples. We will convert this into a taxnomy table to use in MICOM

In [ ]:
abundance = pd.read_csv('../data/raw_data/studyC_original_abundances.csv') # Read in abundances
abundance = abundance[['sample','reads','s']] # Filter table to relevant columns
abundance.rename(columns = {'sample':'sample_id','reads':'abundance','s':'species'}, inplace = True) # Rename for clarity
abundance = abundance.groupby(by = ['sample_id','species']).sum().reset_index() #Sum duplicates
table = pd.pivot_table(abundance,  # Pivot to create abundance matrix
               index = 'sample_id',
               columns = 'species',
               values = 'abundance').fillna(0.0)
table.to_csv('../data/studyC.csv')
abundance['species'] = abundance['species'].str.split('_').str[1].str.lower()
abundance['id'] = abundance['species']
abundance

## Build Models
Now, we'll build our models, with cutoff of 0.001

In [ ]:
manifest = micom.workflows.build(abundance, 
                                 out_folder = '../models/studyC',
                                 model_db = '../agora103_species.qza',
                                 cutoff = 0.001,
                                 threads = 10)

## Load Medium
Load in the carbon-stripped European Diet, and construct the intervention diets by augmenting with inulin and pectin

In [ ]:
medium = pd.read_csv('../media/studyCmedium.csv', index_col = 1)

controlMedium = medium


pectinMedium = pd.concat([medium,
                           pd.DataFrame({'reaction':['EX_pect_m'],
                                         'metabolite':['pect_m'],
                                         'global_id':['EX_pect(e)'],
                                         'flux':[1]
                                        },index = ['EX_pect_m'])])


inulinMedium = pd.concat([medium,
                           pd.DataFrame({'reaction':['EX_inulin_m'],
                                         'metabolite':['inulin_m'],
                                         'global_id':['EX_inulin(e)'],
                                         'flux':[14]
                                        },index = ['EX_inulin_m'])])

inulinMedium

## Grow Models
Now we'll grow the samples using the respective media we constructed

In [ ]:
manifest = pd.read_csv('../models/studyC/manifest.csv')
controlGrowth = micom.workflows.grow(manifest,
                                  model_folder='../models/studyC',
                                  medium = controlMedium, 
                                  tradeoff = 0.7,
                                  strategy ='none', 
                                  threads = 10)

pectinGrowth = micom.workflows.grow(manifest,
                                  model_folder='../models/studyC',
                                  medium = pectinMedium, 
                                  tradeoff = 0.7, 
                                  strategy = 'none', 
                                  threads = 10)

inulinGrowth = micom.workflows.grow(manifest, 
                                  model_folder='../models/studyC',
                                  medium = inulinMedium, 
                                  tradeoff = 0.7, 
                                  strategy = 'none',
                                  threads = 10)

## Compare SCFA Fluxes
We can now extract the production fluxes of SCFAs from each growth simulation, and construct a dataframe with both measured and predicted production rates.

In [ ]:
controlProduction = micom.measures.production_rates(controlGrowth)
controlProduction['treatment'] = 'control'
pectinProduction = micom.measures.production_rates(pectinGrowth)
pectinProduction['treatment'] = 'pectin'
inulinProduction = micom.measures.production_rates(inulinGrowth)
inulinProduction['treatment'] = 'inulin'
production = pd.concat([controlProduction, pectinProduction, inulinProduction])
production['sample_id'] = production['sample_id']+'_'+production['treatment']
res = production[(production['name']==('butyrate'))|
                   (production['name']==('acetate'))|
                   (production['name']==('propionate'))]

res = pd.pivot(res, index = 'sample_id', 
                         columns = 'name',
                         values = 'flux')
res.rename(columns = {'acetate':'acetatePredicted',
                      'butyrate':'butyratePredicted',
                      'propionate':'propionatePredicted'
                      }, inplace = True
          )
res = pd.concat([res, scfa], axis = 1)
res.dropna(inplace = True)
res

## Scale by Biomass
To compare results between studies we will scale the predicted SCFAs by biomass. Without accurate qPCR, we will instead use human reads as proxy. 

In [ ]:
abundance = pd.read_csv('../data/raw_data/studyC_original_abundances.csv')
sampleReads = abundance.groupby('sample')['reads'].sum()
humanReads = abundance[abundance['g']=='Homo'].set_index('sample')['reads']
pctHuman = humanReads/sampleReads                                                
pctHuman = pctHuman.groupby(level = 0).mean().to_dict()
res['pct_human'] = res['donor'].map(pctHuman)
res['acetatePredicted'] = res['acetatePredicted']*(1-res['pct_human'])
res['butyratePredicted'] = res['butyratePredicted']*(1-res['pct_human'])
res['propionatePredicted'] = res['propionatePredicted']*(1-res['pct_human'])

res

## Plot Results
Finally, we'll plot predicted vs measured fluxes against each other

In [ ]:
fig1 = (ggplot(
    res, aes(x = 'butyrateMeasured', y = 'butyratePredicted'))
    +geom_smooth(method = 'lm', linetype = '--')
    +geom_point(aes(color = 'treatment'), size = 8)
    +scale_color_manual(limits = ['control','inulin','pectin'], 
                        values = ['cornflowerblue', 'coral', 'mediumseagreen'],
                        labels = ['Control','Inulin','Pectin'])
    +labs(title='Butyrate',
          x='Measured ($\dfrac{mmol}{L*h}$)',
          y = 'Predicted ($\dfrac{mmol}{gDCW*h}$)',
          color = 'Treatment')
    +theme(
            text = element_text(size=35, color = 'black'),panel_background=element_rect(fill = "white",
                                    colour = "white",size = 0.5, linetype = "solid"),panel_grid= element_blank(),
                                    axis_line = element_line(size = 2, linetype = "solid",colour = "black"),
                                    legend_position='right',axis_text_x=element_text(rotation = 20, hjust = 1))
)
fig1

In [ ]:
scipy.stats.pearsonr(res['butyratePredicted'], res['butyrateMeasured'])

In [ ]:
fig2 = (ggplot(
    res, aes(x = 'propionateMeasured', y = 'propionatePredicted'))
    +geom_smooth(method = 'lm', linetype = '--')
    +geom_point(aes(color = 'treatment'), size = 8)
    +scale_color_manual(limits = ['control','inulin','pectin'], 
                        values = ['cornflowerblue', 'coral', 'mediumseagreen'],
                        labels = ['Control','Inulin','Pectin'])
    +labs(title='Propionate',
          x='Measured ($\dfrac{mmol}{L*h}$)',
          y = 'Predicted ($\dfrac{mmol}{gDCW*h}$)', 
         color = 'Treatment')
    +theme(text = element_text(size=35, color = 'black'),panel_background=element_rect(fill = "white",
                                    colour = "white",size = 0.5, linetype = "solid"),panel_grid= element_blank(),
                                    axis_line = element_line(size = 2, linetype = "solid",colour = "black"),
                                    legend_position='right',axis_text_x=element_text(rotation = 20, hjust = 1))
)
fig2

In [ ]:
scipy.stats.pearsonr(res['propionatePredicted'], res['propionateMeasured'])

## Save all results

In [ ]:
res.to_csv('../results/studyC.csv')